In [17]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import config

print(config.ROOT_FOLDER)

/Users/alexander.prams/autotune


In [15]:
with open(os.path.join(config.ROOT_FOLDER, 'experiment_results/hpo_dataset_optimizer_results.pickle'), 'rb') as handle:
    optimizer_results = pickle.load(handle)

In [15]:
# Optimizer results dict has structure [optimizer.name][classifier_name][dataset_idx][rep_id] -> optimizer output
optimizers = list(optimizer_results.keys())
classifiers = list(optimizer_results[optimizers[0]].keys())
n_datasets = len(optimizer_results[optimizers[0]][classifiers[0]])
n_reps_per_dataset = len(optimizer_results[optimizers[0]][classifiers[0]][0])

print("Optimizers: ", optimizers)
print("Classifiers: ", classifiers)
print("n_datasets: ", n_datasets)
print("n_reps_per_dataset", n_reps_per_dataset)

Optimizers:  ['GridSearch', 'RandomSearch', 'GA', 'TPE']
Classifiers:  ['libsvm_svc', 'liblinear', 'random_forest']
n_datasets:  42
n_reps_per_dataset 10


In [20]:
def analyze_optimizer(optimizer_results, opt_name="GridSearch"):
    opt_results = optimizer_results[opt_name]
    for classifier in opt_results:
        results = np.array(opt_results[classifier])
        print(results.shape)
        

analyze_optimizer(optimizer_results)

(42, 10, 1596, 2)
(42, 10, 168, 2)
(42, 10, 400, 2)
